<a href="https://colab.research.google.com/github/AlineSchaub/DataScience21/blob/7_Block/Kopie_von_Exercise_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Image Classification with CNNs
![CIFAR10](https://miro.medium.com/max/964/1*syyml8q8s1Yt-iEea5m1Ag.png)

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html) is a small toy data set for image classifaction. 

* It has 60k images from ten diffenrent classes (like car, airplane, horse,..)
* The RGB images are of size 32x32 pixels  

In [1]:
#import needed libs
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

#some vars to controll the training -> use them in your code
batch_size = 32
num_classes = 10 #fixed!
epochs = 10

# Get the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Task:
* visualize some sample images from different classes
* build and train the ***AlexNet*** CNN architecture from the lecture, use the following Keras layers:
    * overview: https://keras.io/api/layers/
    * 2d convolution: https://keras.io/api/layers/convolution_layers/convolution2d/
    * Fully Connected (=Matrix Multiplication MLP): https://keras.io/api/layers/core_layers/dense/
    * Pooling: https://keras.io/api/layers/pooling_layers/max_pooling2d/   
* use 'relu' functions for the non-linear activation -> https://keras.io/api/layers/activation_layers/relu/
* optimize the learning hyper-paramters to get the best  test accuracy

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import activations

#RGB (3) und Bildgröße 32 auf 32
inputs = keras.Input(shape=(32, 32, 3))


#Bilder zuschneiden
x_1 = CenterCrop(height=30, width=30)(inputs)


# Rescale images to [0, 1]
x_2 = Rescaling(scale=1.0 / 255)(x_1)

# 2d Convolution layer
x_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x_2)

#fully connected layer
x_4 = layers.Dense(32, activation='relu')(x_3)

#Pooling layer
x_5 = layers.MaxPooling2D(pool_size=(3, 3))(x_4)


x_6 = layers.GlobalAveragePooling2D()(x_5)



In [3]:

outputs = layers.Dense(num_classes, activation=activations.relu)(x_6)

In [4]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
center_crop (CenterCrop)     (None, 30, 30, 3)         0         
_________________________________________________________________
rescaling (Rescaling)        (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        896       
_________________________________________________________________
dense (Dense)                (None, 28, 28, 32)        1056      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0     

In [6]:
#optimize + loss
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

In [7]:
#batch size und epochs von oben

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10
1563/1563 [==============================] - 39s 24ms/step - loss: 7.6660
Epoch 2/10
1563/1563 [==============================] - 37s 24ms/step - loss: 7.3471
Epoch 3/10
1563/1563 [==============================] - 37s 24ms/step - loss: 7.4018
Epoch 4/10
1563/1563 [==============================] - 37s 24ms/step - loss: 7.2778
Epoch 5/10
1563/1563 [==============================] - 37s 24ms/step - loss: 6.0337
Epoch 6/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan
Epoch 8/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan
Epoch 9/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan
Epoch 10/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan


In [8]:
print(history.history)

{'loss': [7.478695392608643, 7.399676322937012, 7.401665687561035, 6.983435153961182, 6.063174247741699, nan, nan, nan, nan, nan]}


In [36]:
#Metrics
model.compile('adam', 'binary_crossentropy', metrics='accuracy')




In [37]:
import tensorflow as tf

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [38]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)


In [39]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model.fit(dataset, epochs=epochs, batch_size=batch_size, validation_data=val_dataset)

1563/1563 [==============================] - 41s 26ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000


In [40]:
loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

313/313 [==============================] - 3s 10ms/step - loss: nan - accuracy: 0.1000
loss: nan
acc: 0.10
